In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Valor -
[nltk_data]     Javis\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Valor -
[nltk_data]     Javis\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict
import string

# abrir o arquivo PDF
with open('RO SIAPE.pdf', 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    # extrair o texto do PDF
    text = ''
    for i in range(len(reader.pages)):
        text += reader.pages[i].extract_text()

# remover pontuações e números do texto
text = text.translate(str.maketrans('', '', string.punctuation+string.digits))

# tokenizar o texto em palavras e sentenças
tokens = word_tokenize(text)
sentences = sent_tokenize(text)

# remover stopwords do texto
stop_words = set(stopwords.words('portuguese'))
filtered_tokens = [word for word in tokens if word.casefold() not in stop_words]

# calcular a frequência de cada palavra no texto
freq_dist = nltk.FreqDist(filtered_tokens)

# criar um dicionário para mapear cada palavra para os parágrafos em que aparece
word_to_paragraphs = defaultdict(list)
for i in range(len(reader.pages)):
    page_text = reader.pages[i].extract_text()
    page_text = page_text.translate(str.maketrans('', '', string.punctuation+string.digits))
    page_sentences = sent_tokenize(page_text)
    page_filtered_tokens = [word for word in word_tokenize(page_text) if word.casefold() not in stop_words]
    for word in page_filtered_tokens:
        for sentence in page_sentences:
            if word in sentence:
                word_to_paragraphs[word].append(i)

# permitir que o usuário faça perguntas
while True:
    question = input('Digite sua pergunta ou digite "sair" para encerrar: ')
    if question.lower() == 'sair':
        break
    else:
        # tokenizar a pergunta em palavras
        question_tokens = word_tokenize(question)

        # remover stopwords da pergunta
        filtered_question = [word for word in question_tokens if word.casefold() not in stop_words]

        # procurar pelos parágrafos que contêm todas as palavras-chave da pergunta
        relevant_paragraphs = []
        for word in filtered_question:
            if word in word_to_paragraphs:
                relevant_paragraphs.append(set(word_to_paragraphs[word]))

        # encontrar a interseção dos parágrafos relevantes
        if relevant_paragraphs:
            common_paragraphs = set.intersection(*relevant_paragraphs)
        else:
            common_paragraphs = set()

        # imprimir o parágrafo relevante
        if common_paragraphs:
            paragraph_text = ''
            for i in common_paragraphs:
                paragraph_text += reader.pages[i].extract_text()
            print('Aqui está o parágrafo relevante para a sua pergunta:\n')
            print(paragraph_text)
        else:
            print('Desculpe, não consegui encontrar nenhuma informação relevante para a sua pergunta.')
